In [1]:
import json

In [2]:
import tensorflow as tf
import numpy as np
import keras

In [3]:
from sklearn.preprocessing import LabelEncoder

In [233]:
with open (mode='r',file='dataset_ner.json') as f :
  data = json.load(f)

In [234]:
data

[{'text': 'Toko Sentosa Jumlah : 3 barang : tablet jenis : KeyMaster harga : 23547138 diskon : 25%',
  'labels': ['barang', 'jenis', 'harga', 'diskon']},
 {'text': 'Pembelian di Medan 17/06/2025, produk: smartphone, merek: MouseTech, harga: 6293405, kondisi: baru',
  'labels': ['merek', 'harga']},
 {'text': 'Pesanan masuk: 93 unit printer VisionCam, kondisi rekondisi, harga Rp12742855, diskon 20%',
  'labels': ['harga', 'diskon']},
 {'text': 'Pesanan masuk: 63 unit smartphone Ai Laptop, kondisi rusak ringan, harga Rp5718854, diskon 25%',
  'labels': ['harga', 'diskon']},
 {'text': 'Pesanan masuk: 20 unit tablet KeyMaster, kondisi rekondisi, harga Rp24274299, diskon 25%',
  'labels': ['harga', 'diskon']},
 {'text': 'Toko Sentosa Jumlah : 79 barang : monitor jenis : TechPro harga : 1620350 diskon : 25%',
  'labels': ['barang', 'jenis', 'harga', 'diskon']},
 {'text': 'Toko Sentosa Jumlah : 50 barang : mouse jenis : KeyMaster harga : 12448204 diskon : 25%',
  'labels': ['barang', 'jenis', 

In [5]:
texts = list()
label = list()
for i in data  :
  d = i
  texts.append(i['text'])
  label.append(i['labels'])

In [6]:
labels = list()
for i in label :
  for j in range(len(i)) :
    labels.append(i[j])

In [7]:
Enc_label = LabelEncoder().fit(labels)

In [8]:
y_train = list()
Y_rule = list()
for i in range(len(label)) :
  y_train.append(Enc_label.transform(label[i]) + 1)
  Y_rule.append(Enc_label.transform(label[i]) + 1)

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
y_train =pad_sequences(y_train,maxlen=4,padding='post')

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [12]:
x_train = tokenizer.texts_to_sequences(texts)

In [13]:
vocab_size = len(tokenizer.index_word) + 1

In [14]:
x_train = pad_sequences(x_train,maxlen=15,padding='post')

In [15]:
Rule_label = list()
for i in Y_rule :
  Rule_label.append(len(i))

In [16]:
Y_label_Rule = np.array(Rule_label)

In [17]:
np.unique(Y_label_Rule)

array([2, 3, 4])

In [18]:
inputs_ = keras.Input(shape=(15,))
Dense1 = keras.layers.Dense(32,activation='relu')(inputs_)
Dense2 =keras.layers.Dense(16,activation='relu')(Dense1)
outputs = keras.layers.Dense(5,activation='softmax')(Dense2)

In [19]:
Rule_Manager_Models = keras.Model(inputs_,outputs)

In [20]:
Rule_Manager_Models.compile(loss= keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])

In [21]:
Rule_Manager_Models.fit(x_train,Y_label_Rule,epochs=8,verbose=2)

Epoch 1/8
313/313 - 2s - 7ms/step - accuracy: 0.7997 - loss: 30.7288
Epoch 2/8
313/313 - 1s - 2ms/step - accuracy: 0.9421 - loss: 0.5932
Epoch 3/8
313/313 - 1s - 2ms/step - accuracy: 0.9477 - loss: 0.9193
Epoch 4/8
313/313 - 1s - 2ms/step - accuracy: 0.9671 - loss: 0.5239
Epoch 5/8
313/313 - 1s - 2ms/step - accuracy: 0.9657 - loss: 0.7676
Epoch 6/8
313/313 - 1s - 2ms/step - accuracy: 1.0000 - loss: 1.7162e-04
Epoch 7/8
313/313 - 1s - 2ms/step - accuracy: 1.0000 - loss: 6.1406e-05
Epoch 8/8
313/313 - 1s - 2ms/step - accuracy: 1.0000 - loss: 5.2276e-05


In [23]:
result=Rule_Manager_Models.predict(x_train)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [24]:
res = list()
for _ in result :
  res.append(np.argmax(_))

In [25]:
test = list()
for i in range(len(res)) :
  if res[i] == Y_label_Rule[i] :
    test.append('True')
  else :
    test.append('False')

In [232]:
class Rule_based_Layers (keras.Model) :
  def __init__ (self,vocab_size,output_dim,d_model) :
    super(Rule_based_Layers,self).__init__()
    self.Embedding = keras.layers.Embedding(input_dim=vocab_size,output_dim=output_dim)
    self.self_attention_rule1 = keras.layers.Attention()
    self.self_attention_rule2 = keras.layers.Attention()
    self.self_attention_rule3 = keras.layers.Attention()
    self.self_attention_rule4 = keras.layers.Attention()
    self.Lstm_Rule1 = keras.layers.LSTM(64,return_sequences=True)
    self.Lstm_Rule2 = keras.layers.LSTM(64,return_sequences=True)
    self.Lstm_Rule3 = keras.layers.LSTM(64,return_sequences=True)
    self.Lstm_Rule4 = keras.layers.LSTM(64,return_sequences=True)
    self.Query_Keys_Values_Rule1 = [keras.layers.Dense(d_model) for _ in range(3)]
    self.Query_Keys_Values_Rule2 = [keras.layers.Dense(d_model) for _ in range(3)]
    self.Query_Keys_Values_Rule3 = [keras.layers.Dense(d_model) for _ in range(3)]
    self.Query_Keys_Values_Rule4 = [keras.layers.Dense(d_model) for _ in range(3)]
    self.Rule_models_Manager = Rule_Manager_Models
    self.Normal_layer = [keras.layers.LayerNormalization()for _ in range(4)]
    self.Global_pool = [keras.layers.GlobalAveragePooling1D() for _ in range(4)]
    self.Dense_prob = [keras.layers.Dense(256,activation='relu') for _ in range(4)]
    self.F_layers = [keras.layers.Dense(9,activation='softmax') for _ in range(4)]

  def get_rule_maneger (self,x,Model) :
    predicted = Model.predict(x)
    list_true = list()
    for i in range(len(predicted)) :
      list_true.append(np.argmax(predicted[i]))
    rule1 = list()
    rule1_temp = list()
    rule2 = list()
    rule2_temp = list()
    rule3 = list()
    rule3_temp = list()
    rule4 = list()
    rule4_temp = list()
    for i in range(len(list_true)) :
      if list_true[i] == 1 :
        for _ in range(list_true[i]):
          rule1_temp.append(x[i])
        rule1.append(rule1_temp)
        rule1_temp = list()
      elif list_true[i] == 2 :
        for _ in range(list_true[i]):
          rule2_temp.append(x[i])
        rule2.append(rule2_temp)
        rule2_temp = list()
      elif list_true[i] == 3 :
        for _ in range(list_true[i]):
          rule3_temp.append(x[i])
        rule3.append(rule3_temp)
        rule3_temp = list()
      else :
        for _ in range(list_true[i]) :
          rule4_temp.append(x[i])
        rule4.append(rule4_temp)
        rule4_temp = list()
    rule1 = np.array(rule1)
    rule2 = np.array(rule2)
    rule3 = np.array(rule3)
    rule4 = np.array(rule4)
    return rule1,rule2,rule3,rule4

  def Rule_data_schedule (self,x,Model) :
    rule1,rule2,rule3,rule4 = self.get_rule_maneger(x,Model)
    dat_rule1 = list()
    dat_rule2 = list()
    dat_rule3 = list()
    dat_rule4 = list()
    if len(rule1) != 0 :
      for i in rule1 :
        dat_rule1.append(i)
    if len(rule2) != 0 :
      x1 = list()
      x2 = list()
      for i in range(len(rule2)) :
        x1.append(rule2[i][0])
        x2.append(rule2[i][1])
      dat_rule2.append(x1)
      dat_rule2.append(x2)
    if len(rule3) != 0:
      x1 = list()
      x2 = list()
      x3 = list()
      for i in range(len(rule3)) :
        x1.append(rule3[i][0])
        x2.append(rule3[i][1])
        x3.append(rule3[i][2])
      dat_rule3.append(x1)
      dat_rule3.append(x2)
      dat_rule3.append(x3)
    if len(rule4) != 0 :
      x1 = list()
      x2 = list()
      x3 = list()
      x4 = list()
      for i in range(len(rule4)) :
        x1.append(rule4[i][0])
        x2.append(rule4[i][1])
        x3.append(rule4[i][2])
        x4.append(rule4[i][3])
      dat_rule4.append(x1)
      dat_rule4.append(x2)
      dat_rule4.append(x3)
      dat_rule4.append(x4)
    return np.array(dat_rule1),np.array(dat_rule2),np.array(dat_rule3),np.array(dat_rule4)

  def Rule_1 (self,x) :
    x = self.Embedding(x)
    Query  = self.Query_Keys_Values_Rule1[0](x)
    Keys = self.Query_Keys_Values_Rule1[1](x)
    Value  = self.Query_Keys_Values_Rule1[2](x)
    x = self.self_attention_rule1([Query,Keys,Value])
    x = self.Lstm_Rule1(x)
    x = self.Normal_layer[0](x)
    x = self.Global_pool[0](x)
    x = self.Dense_prob[0](x)
    x = self.F_layers[0](x)
    return x

  def Rule_2 (self,x) :
    x = self.Embedding(x)
    Query = self.Query_Keys_Values_Rule2[0](x)
    Keys = self.Query_Keys_Values_Rule2[1](x)
    Value = self.Query_Keys_Values_Rule2[2](x)
    x = self.self_attention_rule2([Query,Keys,Value])
    x = self.Lstm_Rule2(x)
    x = self.Normal_layer[1](x)
    x = self.Global_pool[1](x)
    x = self.Dense_prob[1](x)
    x = self.F_layers[1](x)
    return x

  def Rule_3 (self,x) :
    x = self.Embedding(x)
    Query = self.Query_Keys_Values_Rule3[0](x)
    Keys = self.Query_Keys_Values_Rule3[1](x)
    Values = self.Query_Keys_Values_Rule3[2](x)
    x = self.self_attention_rule3([Query,Keys,Values])
    x = self.Lstm_Rule3(x)
    x = self.Normal_layer[2](x)
    x = self.Global_pool[2](x)
    x = self.Dense_prob[2](x)
    x = self.F_layers[2](x)
    return x

  def Rule_4 (self,x) :
    x = self.Embedding(x)
    Query = self.Query_Keys_Values_Rule4[0](x)
    Keys = self.Query_Keys_Values_Rule4[1](x)
    Values = self.Query_Keys_Values_Rule4[2](x)
    x = self.self_attention_rule4([Query,Keys,Values])
    x = self.Lstm_Rule4(x)
    x = self.Normal_layer[3](x)
    x = self.Global_pool[3](x)
    x = self.Dense_prob[3](x)
    x = self.F_layers[3](x)
    return x

  def call(self,x) :
    Model = self.Rule_models_Manager
    all_result = list()
    sub_result = list()
    x_rule1,x_rule2,x_rule3,x_rule4 = self.Rule_data_schedule(x,Model)
    Rule_result = list()
    Rule2_result = list()
    Rule3_result = list()
    Rule4_result = list()
    if len(x_rule1) != 0 :
      R1 = self.Rule_1(x_rule1)
      Rule_result.append(R1)
      Rule_result = np.array(Rule_result)
    if len(x_rule2) != 0 :
      R1 = self.Rule_1(x_rule2[0])
      sub_result.append(R1)
      R2 = self.Rule_2(x_rule2[1])
      sub_result.append(R2)
      Rule2_result.append(sub_result)
      Rule2_result = np.array(Rule2_result)
      sub_result = list()
    if len(x_rule3) != 0 :
      R1 = self.Rule_1(x_rule3[0])
      sub_result.append(R1)
      R2 = self.Rule_2(x_rule3[1])
      sub_result.append(R2)
      R3 = self.Rule_3(x_rule3[2])
      sub_result.append(R3)
      Rule3_result.append(sub_result)
      Rule3_result = np.array(Rule3_result)
      sub_result = list()
    if len(x_rule4) != 0 :
      R1 = self.Rule_1(x_rule4[0])
      sub_result.append(R1)
      R2 =self.Rule_2(x_rule4[1])
      sub_result.append(R2)
      R3 = self.Rule_3(x_rule4[2])
      sub_result.append(R3)
      R4 = self.Rule_4(x_rule4[3])
      sub_result.append(R4)
      Rule4_result.append(sub_result)
      Rule4_result = np.array(Rule4_result)
      sub_result = list()
    if len(Rule_result) != 0 :
      c = 0
      for i in range(len(Rule_result)):
        for i in range(1) :
          sub_result.append(tf.argmax(Rule_result[c]))
          sub_result.append(0)
          sub_result.append(0)
          sub_result.append(0)
        all_result.append(sub_result)
        sub_result = list()
        c+=1
    if len(Rule2_result) != 0:
      c = 0
      for i in range(len(Rule2_result[0])) :
        for j in range(1):
          sub_result.append(tf.argmax(Rule2_result[0][0][c]))
          sub_result.append(tf.argmax(Rule2_result[0][1][c]))
          sub_result.append(0)
          sub_result.append(0)
        all_result.append(sub_result)
        sub_result = list()
        c+=1
    if len(Rule3_result) != 0 :
      c = 0
      for i in range(len(Rule3_result[0])) :
        for j in range(1) :
          sub_result.append(tf.argmax(Rule3_result[0][0][c]))
          sub_result.append(tf.argmax(Rule3_result[0][1][c]))
          sub_result.append(tf.argmax(Rule3_result[0][2][c]))
          sub_result.append(0)
        all_result.append(sub_result)
        sub_result = list()
        c+=1
    if len(Rule4_result) != 0 :
      c = 0
      for i in range(len(Rule4_result[0])) :
        for j in range(1) :
          sub_result.append(tf.argmax(Rule4_result[0][0][c]))
          sub_result.append(tf.argmax(Rule4_result[0][1][c]))
          sub_result.append(tf.argmax(Rule4_result[0][2][c]))
          sub_result.append(tf.argmax(Rule4_result[0][3][c]))
        all_result.append(sub_result)
        sub_result = list()
        c+=1
    all_result = np.array(all_result)
    return all_result

In [225]:
Rule_layers = Rule_based_Layers(vocab_size=vocab_size,output_dim=128,d_model=128)

In [226]:
data = Rule_layers(x_train)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [230]:
x_train

array([[ 17,  18,  19, ...,   0,   0,   0],
       [ 22,   6,  55, ...,   5,  25,   0],
       [ 13,  14, 123, ...,   0,   0,   0],
       ...,
       [  7,  10,   8, ...,   9,  32,   0],
       [  7,  44,   8, ...,  25,   0,   0],
       [ 11,  12,   3, ...,   0,   0,   0]], dtype=int32)

In [231]:
Y_label_Rule.sort

array([4, 2, 2, ..., 4, 4, 3])

In [199]:
Rule_layers.summary()

Model: "rule_based__layers_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_22 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_88 (Attention)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_89 (Attention)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_90 (Attention)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_91 (Attention)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_88 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_89 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_90 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_91 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_428 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_429 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_430 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_431 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_432 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_433 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_434 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_435 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_436 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_437 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_438 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_439 (Dense)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,125 (4.39 KB)

 Trainable params: 1,125 (4.39 KB)

 Non-trainable params: 0 (0.00 B)

In [181]:
y_train

array([[1, 4, 3, 2],
       [7, 3, 0, 0],
       [3, 2, 0, 0],
       ...,
       [7, 3, 6, 5],
       [7, 3, 6, 5],
       [3, 2, 6, 0]], dtype=int32)

In [188]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(1)

In [189]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 15), dtype=tf.int32, name=None), TensorSpec(shape=(None, 4), dtype=tf.int32, name=None))>

In [179]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.AdamW(learning_rate=0.01)
metrics = keras.metrics.BinaryAccuracy()

In [245]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, Dropout, Dense, LayerNormalization, MultiHeadAttention, Input
from tensorflow.keras.models import Model

class TransformerEncoderLayer(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.norm2(out1 + ffn_output)

class TransformerEncoder(Layer):  # Ubah dari Model ke Layer
    def __init__(self, vocab_size, max_len, embed_dim, num_heads, ff_dim, num_layers, dropout_rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_embedding = self.add_weight(shape=(1, max_len, embed_dim), initializer="zeros", trainable=True)
        self.enc_layers = [TransformerEncoderLayer(embed_dim, num_heads, ff_dim, dropout_rate) for _ in range(num_layers)]
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):  # Default training=False
        seq_len = tf.shape(inputs)[1]
        x = self.embedding(inputs)
        x += self.pos_embedding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for enc_layer in self.enc_layers:
            x = enc_layer(x, training=training)  # Tambahkan training
        return x

# Hyperparameters
VOCAB_SIZE = 10000
MAX_LEN = 15
EMBED_DIM = 128
NUM_HEADS = 4
FF_DIM = 512
NUM_LAYERS = 4

# Model initialization
x_input = Input(shape=(MAX_LEN,), dtype=tf.int32)
encoder = TransformerEncoder(VOCAB_SIZE, MAX_LEN, EMBED_DIM, NUM_HEADS, FF_DIM, NUM_LAYERS)
x_output = encoder(x_input)  # Tidak perlu training=False karena sudah diatur default

# Build model
model = Model(inputs=x_input, outputs=x_output)
model.summary()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_34 (InputLayer)          │ (None, 15)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_7                │ (None, 15, 128)             │       2,866,048 │
│ (TransformerEncoder)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,866,048 (10.93 MB)

 Trainable params: 2,866,048 (10.93 MB)

 Non-trainable params: 0 (0.00 B)

In [246]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [247]:
model.fit(x_train,y_train,epochs=100,verbose=1)

Epoch 1/100


ValueError: Arguments `target` and `output` must have the same shape up until the last dimension: target.shape=(None, 4), output.shape=(None, 15, 128)